In [ ]:
import random
import sys
from ImportantConfig import Config
config = Config()
from sql2fea import TreeBuilder,value_extractor
from NET import TreeNet
from sql2fea import Sql2Vec
from TreeLSTM import SPINN

sys.stdout = open(config.log_file, "w")
random.seed(113)
with open(config.queries_file) as f:
    import json
    queries = json.load(f)

tree_builder = TreeBuilder()
sql2vec = Sql2Vec()
value_network = SPINN(head_num=config.head_num, input_size=7+2, hidden_size=config.hidden_size, table_num = 50,sql_size = 40*40+config.max_column).to(config.device)
for name, param in value_network.named_parameters():
    from torch.nn import init
    if len(param.shape)==2:
        init.xavier_normal(param)
    else:
        init.uniform(param)


treenet_model = TreeNet(tree_builder, value_network)

mask = (torch.rand(1,config.head_num,device = config.device)<0.9).long()

# 训练模型
for epoch in range(num_epochs):
    total_loss = 0.0
    for batch_data in train_dataloader:
        sql_vec,alias = sql2vec.to_vec(sql)
        plan_json, target_value = batch_data
        loss, mean, variance, exp_variance = treenet_model.train(plan_json, sql_vec, target_value, mask, is_train=True)
        total_loss +=loss
    # 打印或记录训练过程中的损失等信息
    avg_loss = total_loss / len(train_data)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

# 预测
with torch.no_grad():
    for test_batch_data in test_dataloader:
        plan_json, sql_vec, target_value, mask = test_batch_data
        _, mean, _, exp_variance = treenet_model.train(plan_json, sql_vec, target_value, mask, is_train=False)   # mean 变量来获取模型对于执行计划的预测平均值
        # 使用预测结果进行后续操作，例如输出预测值或进行评估
